## Quiz #0502 (Solution)

### "Human Activity Recognition" 

#### Answer the following questions by providing Python code:
#### Objectives:
- Carry out the EDA.
- Carry out the data pre-processing.
- Optimize and test a predictive model of your choice.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

#### Read in data:
The explanation on the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).

In [ ]:
# Go to the directory where the data file is located. 
# os.chdir(r'~~')                # Please, replace the path with your own. 

In [ ]:
df = pd.read_csv('data_human activity recognition.csv', header='infer')

In [ ]:
df.shape

In [ ]:
df.columns

1). Carry out the EDA. Check for the missing values. HINT: The response variable is 'classe'.

In [ ]:
# Descriptive statistics of the numeric variables.
df.describe()

In [ ]:
# Frequency table of the response variable.
freqs = df.classe.value_counts()
sns.barplot(freqs.index, freqs.values,ci=None)
plt.show()

In [ ]:
# Proportion of the missing values per column.
pd.isnull(df).mean(axis=0)

2). Get rid of the columns that have more than 97% missing values.

In [ ]:
# Keep only those columns with less than 97% missing value.
isnotZero = np.array(pd.isnull(df).mean(axis=0) < 0.97)
df = df.loc[:, isnotZero]

In [ ]:
# Now the verify the result. No more missing values!
pd.isnull(df).mean(axis=0)

In [ ]:
# Check the size.
df.shape

3). Get rid of the unnecessary columns. HINT: Those columns with "time" in the name and those that are obviously unnecessary.

In [ ]:
# Get rid of columns with "time" in the name.
header =df.columns
isnotTime = []
for col in header:
    if "time" in col:
        isnotTime.append(False)
    else:
        isnotTime.append(True)
df = df.loc[:, isnotTime]

# Get rid of other unnecessary columns.
df = df.drop(columns = ['Unnamed: 0','user_name','new_window'])
print(df.shape)
df.head(5)

4). Label encode the responde variable. HINT: use preprocessing.LabelEncoder(). 

In [ ]:
LE = preprocessing.LabelEncoder()
X = np.array(df.drop(columns='classe'))
Y = np.array(df.loc[:,'classe'])
Y = LE.fit_transform(Y)

5). Carry out min-max scaling of the exploratory variables. HINT: use preprocessing.MinMaxScaler().

In [ ]:
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)

6). Choose an algorithm and carry out the predictive analysis.

- Optimize the hyperparameter(s)
- Calculate the accuracy.
- Aim for upper 90% accuracy.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

In [ ]:
# Here, we will apply the KNN.
# Parameter grid.
k_grid = np.arange(1,11,1)
parameters = {'n_neighbors':k_grid}

In [ ]:
# Optimize the k.
gridCV = GridSearchCV(KNeighborsClassifier(), parameters, cv=10, n_jobs = -1)       # "n_jobs = -1" means "use all the CPU cores".
gridCV.fit(X_train, Y_train)
best_k = gridCV.best_params_['n_neighbors']
print("Best k : " + str(best_k))

In [ ]:
# Test with the best k.
KNN_best = KNeighborsClassifier(n_neighbors=best_k)
KNN_best.fit(X_train, Y_train)
Y_pred = KNN_best.predict(X_test)
print( "Best KNN accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))